In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
BasePath = '/content/drive/My Drive/NLP-Resources/articleRefs.v2525'
ArticleFile = F'{BasePath}/../Devopedia/devopediaArticles.v2525.json'
MetaFile = F'{BasePath}/meta.v2525.jl'
RefPath = 'refs'

In [ ]:
import re
import os
import pandas as pd

In [ ]:
# Creates folder output: rename
!tar zxf "$BasePath"/articleRefs.txt.tar.gz
!mv output $RefPath

In [ ]:
articles = pd.read_json(ArticleFile, orient='index')

In [ ]:
# Expand secs column to get References column
secs = articles['secs'].apply(pd.Series)
articles = pd.concat([articles, secs], axis=1)
articles = articles[~articles.References.isna()]

In [ ]:
articles.head()

In [ ]:
def convert_article_refs(refs):
    return tuple(ref['url'] if 'url' in ref else None for ref in refs)

In [ ]:
articles['urls'] = articles['References'].apply(convert_article_refs)

In [ ]:
articles.columns

In [ ]:
refs = pd.read_json(MetaFile, lines=True)
refs = refs[refs.status==200]
refs.head()

In [ ]:
def copy_article_ref_files(urls, alias, srcpath):
    ''' Create a folder for each article and 
    copy that article's references into that folder.
    '''
    files = refs[refs['req_url'].isin(urls)]['fname'].apply(
                lambda x : re.sub(r'\..*?$', r'.txt', x) # we deal with only txt files
            ).tolist()
    
    if not files:
        print(F"No reference files found for article {alias}")
        return

    for f in files:
        dstpath = '{}/{}'.format(srcpath, alias)
        os.makedirs(dstpath, exist_ok=True)
        try:
            os.remove(F'{dstpath}/{f}')
        except FileNotFoundError:
            pass
        if os.path.exists(F'{RefPath}/{f}'):
            os.symlink(F'../{f}', F'{dstpath}/{f}')

In [ ]:
articles.apply(lambda x : copy_article_ref_files(x.urls, x.alias, RefPath), axis=1)

In [ ]:
# Exclude articles in case they give trouble during processing
# articles = articles[~articles.index.isin([...comma-separated list of article IDs...])]